### IS Proof of concept - DRMABS

In [1]:
import numpy as np
import pandas as pd
import itertools
import math
import string
import re
import pymongo

In [2]:
import time
from datetime import datetime
today = datetime.now()

In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
# pip install selenium
from selenium import webdriver

In [4]:
from IPython.display import clear_output

In [5]:
# Files definition
root_path = 'Datasource'
urls_past_file,comment_past_file = [],[]
urls_past_file.append(root_path+"/urls_meiji.csv")
urls_past_file.append(root_path+"/urls_dutchmilk.csv")
urls_past_file.append(root_path+"/urls_dairyhome.csv")
urls_past_file.append(root_path+"/urls_chokchai.csv")
urls_past_file.append(root_path+"/urls_foremost.csv")

comment_past_file.append(root_path+"/comment_DDCF.csv")

# จำนวนกระทู้ที่ต้องการให้ดึงใน 1 นาที
postNum = 7
postNumSleep = math.ceil(60/postNum)

In [6]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["NIDA_PPSN_PRD"]
col_comment = mydb["NIDA_PPSN_SCRAPE"]

### กรณียังไม่เคยดึง URL ที่มีการพูดถึง Keyword ใน Pantip

Get list of URL from search page

Scroll down to get all data (Crawler)

Extract all URLs (web scraping)

Save URLs into csv file

### กรณีเคยเก็บ URL แล้ว ให้เริ่มตรงนี้ได้เลย เพื่อไม่ให้ต้องทำ Request บ่อยๆ

Loop for scaping by URLs from file

In [7]:
df_URLRead = pd.read_csv(urls_past_file[0])
df_URLRead1 = pd.read_csv(urls_past_file[1])
df_URLRead2 = pd.read_csv(urls_past_file[2])
df_URLRead3 = pd.read_csv(urls_past_file[3])
df_URLRead4 = pd.read_csv(urls_past_file[4])
dfURL = pd.concat([df_URLRead,df_URLRead1,df_URLRead2,df_URLRead3,df_URLRead4])
dfURL.drop_duplicates(keep='last',inplace=True,ignore_index=True)
dfURL.columns = ['URLs','Retrived_date']

In [8]:
dfURL.shape

(1522, 2)

In [12]:
def create_ListofDict_InsertMany(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
                "URLs": lx,
                "headline" : df.headline[c],
                "text": df.text[c],
                "article_date" : datetime.strptime(df.article_date[c], '%m/%d/%Y %H:%M:%S'),
                "Retrived_date": df.Retrived_date[c]
        }
        listofdict.append(info)
    return listofdict

In [38]:
startResult = 0
endResult = dfURL.shape[0]
#endResult = 6

In [39]:
comments_list,time_list,error_list = [],[],[]
#col_comment.delete_many({})

for cnt,au in enumerate(dfURL.URLs[startResult:endResult],0):
    today = datetime.now()
    print('Processing URL no. ',cnt+1,' on ',today)
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(au)
    content=driver.page_source
    soup=BeautifulSoup(content,'lxml')
    try:
        div1 = soup.find("div", class_=lambda value: value and value=="display-post-status-leftside")
        headline_t = div1.find("h2", class_=lambda value: value and value=="display-post-title")
        for abbr in soup.find_all("abbr", class_=lambda value: value and value=="timeago"):
            time_list.append(abbr['data-utime'])
        for div3 in soup.find_all("div", class_=lambda value: value and value=="display-post-story"):
            if len(div3['class'])<2:
                comments_list.append([au,headline_t.text.strip(),str(div3.text).strip(),today])
        df_comments = pd.DataFrame(comments_list, columns=['URLs','headline', 'text', 'Retrived_date'])
        df_comments.insert(2,'article_date',value=time_list[0])
        process_data = create_ListofDict_InsertMany(df_comments)
        print(process_data)
        col_comment.insert_many(process_data)
        del df_comments
        comments_list.clear()
        time_list.clear()
    except Exception as err:
        print('กระทู้ไม่ถูกดึงเข้าระบบ', err)
        error_list.append([cnt,au,err])
    time.sleep(postNumSleep)
    driver.quit()
    clear_output(wait=True)

Processing URL no.  1522  on  2020-11-29 07:33:59.076804
[{'URLs': 'https://pantip.com/topic/30011023', 'headline': 'Fit and Firm to Slim ครั้งที่ 21"New Year New YOU!!! ปีใหม่ ไฉไลกว่าเดิม 4/15 เดือน สุดท้าย', 'text': 'แตกกระทู้มาจาก www.pantip.com/topic/30007146/\n\nเปิดบ้านจ้า \n\n\n\nส่งรูปวันปิดเฟสนะจ๊ะ วันที่ 15/1/56 เหลือเวลาแค่อีก 11วันเอง เอาฟิตๆๆๆกันน๊าาา\n\n\n\n    แก้ไขข้อความเมื่อ 4 มกราคม 2556 เวลา 07:03 น.', 'article_date': datetime.datetime(2013, 1, 4, 7, 3, 18), 'Retrived_date': Timestamp('2020-11-29 07:33:59.076804')}, {'URLs': 'https://pantip.com/topic/30011023', 'headline': 'Fit and Firm to Slim ครั้งที่ 21"New Year New YOU!!! ปีใหม่ ไฉไลกว่าเดิม 4/15 เดือน สุดท้าย', 'text': 'รายงานตัวให้ตอบกลับความเห็นนี้ (ใต้ข้อความมีคำว่า ตอบกลับ กดตรงนั้นแล้วจะมีกล่องขึ้นมาให้พิมพ์ค่ะ)\nส่วนเม้าท์มอยให้ตอบที่กล่องข้อความด้านล่างของกระทู้ตามปกติค่ะ', 'article_date': datetime.datetime(2013, 1, 4, 7, 3, 18), 'Retrived_date': Timestamp('2020-11-29 07:33:59.076804')}, {'URLs': 'https

In [40]:
print(error_list)

[[122, 'https://pantip.com/topic/37933589', AttributeError("'NoneType' object has no attribute 'find'")], [1237, 'https://pantip.com/topic/30879221', AttributeError("'NoneType' object has no attribute 'find'")], [1296, 'https://pantip.com/topic/37105528', AttributeError("'NoneType' object has no attribute 'find'")], [1297, 'https://pantip.com/topic/37610087', AttributeError("'NoneType' object has no attribute 'find'")], [1299, 'https://pantip.com/topic/37364633', AttributeError("'NoneType' object has no attribute 'find'")], [1325, 'https://pantip.com/topic/36092354', AttributeError("'NoneType' object has no attribute 'find'")]]
